In [ ]:
!huggingface-cli login

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio

In [ ]:
!pip install peft

In [ ]:
!pip install -q datasets

In [ ]:
!pip install langchain faiss-cpu sentence-transformers transformers


In [ ]:
!pip install PyPDF2

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


# Data structure and Memory System

In [ ]:
# ========================================================================
# STEP 1: Data Structures and Memory System
# ========================================================================
import json
import os
from datetime import datetime, timedelta
from typing import Dict, List, Optional
import pandas as pd
import numpy as np

class UserProfile:
    """Store user's basic info, targets, and preferences"""

    def __init__(self, user_id: str):
        self.user_id = user_id
        self.created_date = datetime.now().strftime('%Y-%m-%d')

        # Default targets (can be customized per user)
        self.targets = {
            'daily_steps': 10000,
            'weekly_zone_minutes': 150,
            'daily_sleep_hours': 8,
            'weekly_exercise_sessions': 3,
            'daily_calories': 2000,
            'daily_whole_grain': 14,  #grams
            'daily_refined_grain': 14, # grams
            'weekly_legumes': 5,       # grams
            'weekly_nuts': 14, #grams
            'weekly_red_meat': 7,          # grams
            'weekly_white_meat': 14,#grams
            'weekly_fruits': 21,       # grams
            'weekly_vegetables': 35,   # grams
            'weekly_fish': 28,       # grams
            'weekly_eggs': 7,    # grams
            'weekly_dairy': 56, # 8 pieces
            'weekly_sweet': 56 # pices
        }

        # User preferences and constraints
        self.preferences = {
            'preferred_exercises': [],
            'food_restrictions': [],
            'food_allergies': [],
            'schedule_constraints': {},
            'health_conditions': []
        }

        # Learning data
        self.response_patterns = {
            'follows_exercise_recs': 0.5,  # How often they follow exercise advice
            'follows_nutrition_recs': 0.5,
            'follows_sleep_recs': 0.5,
            'preferred_rec_types': [],
            'preferred_foods': [],       # Track which food suggestions user likes
            'avoided_foods': []
        }

    def update_targets(self, new_targets: Dict):
        """Update user targets"""
        self.targets.update(new_targets)

    def update_preferences(self, new_prefs: Dict):
        """Update user preferences"""
        self.preferences.update(new_prefs)

    def to_dict(self):
        return {
            'user_id': self.user_id,
            'created_date': self.created_date,
            'targets': self.targets,
            'preferences': self.preferences,
            'response_patterns': self.response_patterns
        }

class WeeklyHealthData:
    """Store one week's health data"""

    def __init__(self, user_id: str, week_start: str, data: Dict):
        self.user_id = user_id
        self.week_start = week_start
        self.week_number = self._calculate_week_number(week_start)

        # Raw weekly data
        self.total_steps = data.get('total_steps', 0)
        self.zone_minutes = data.get('zone_minutes', 0)
        self.sleep_hours = data.get('sleep_hours', [])  # Daily values
        self.exercise_sessions = data.get('exercise_sessions', [])
        self.mood_scores = data.get('mood_scores', [])
        self.food_entries = data.get('food_entries', []) # Store as list of food entries

        # Calculate food category totals from entries
        self.weekly_food_totals = self._calculate_weekly_food_totals()

        # Calculated metrics
        self.avg_daily_steps = self.total_steps / 7
        self.avg_sleep = np.mean(self.sleep_hours) if self.sleep_hours else 0
        self.avg_mood = np.mean(self.mood_scores) if self.mood_scores else 0
        self.total_calories = sum([entry.get('calories', 0) for entry in self.food_entries])
        self.avg_daily_calories = self.total_calories / 7 if self.food_entries else 0

        # Food averages (calculated from weekly totals)
        self.daily_food_averages = {k: v / 7 for k, v in self.weekly_food_totals.items()}


        # Timestamp
        self.created_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    def _calculate_week_number(self, week_start: str) -> int:
        """Calculate week number since user started"""
        week_date = datetime.strptime(week_start, '%Y-%m-%d')
        start_of_year = datetime(week_date.year, 1, 1)
        return ((week_date - start_of_year).days // 7) + 1

    def _calculate_weekly_food_totals(self) -> Dict:
        """Aggregate food entries by category"""
        totals = {
             'whole_grain': 0,  #grams
             'refined_grain': 0, # grams
             'legumes': 0,       # grams
             'nuts': 0, #grams
             'red_meat': 0,          # grams
             'white_meat': 0,#grams
             'fruits': 0,       # grams
             'vegetables': 0,   # grams
             'fish': 0,       # grams
             'eggs': 0,    # grams
             'dairy': 0, # 8 pieces
             'sweet': 0 # pices
        }
        for entry in self.food_entries:
            category = entry.get('category')
            quantity = entry.get('quantity', 0)
            if category and category in totals:
                totals[category] += quantity
        # Prefix keys with 'weekly_' for consistency with targets
        return {f'weekly_{k}': v for k, v in totals.items()}


    def get_food_summary(self) -> str:
        """Get a readable summary of food intake"""
        summary = "Weekly Food Intake:\n"
        food_categories = {
            'dairy': 'Dairy',
            'legumes': 'Legumes',
            'red_meat': 'Red Meat',
            'white_meat': 'White Meat',
            'fish': 'Fish',
            'eggs': 'Eggs',
            'fruits': 'Fruits',
            'vegetables': 'Vegetables',
            'whole_grain': 'Whole Grain',
            'refined_grain': 'Refined Grain',
            'nuts': 'Nuts/Seeds',
            'sweet': 'Sweets'
        }

        for category, display_name in food_categories.items():
            weekly_total = self.weekly_food_totals.get(f'weekly_{category}', 0)
            daily_avg = self.daily_food_averages.get(f'weekly_{category}', 0) # Use weekly total key for average
            summary += f"• {display_name}: {weekly_total:.1f} total ({daily_avg:.1f}/day)\n"

        return summary

    def to_dict(self):
        return {
            'user_id': self.user_id,
            'week_start': self.week_start,
            'week_number': self.week_number,
            'total_steps': self.total_steps,
            'zone_minutes': self.zone_minutes,
            'sleep_hours': self.sleep_hours,
            'exercise_sessions': self.exercise_sessions,
            'mood_scores': self.mood_scores,
            'food_entries': self.food_entries, # Save raw entries
            'weekly_food_totals': self.weekly_food_totals, # Save calculated totals
            'avg_daily_steps': self.avg_daily_steps,
            'avg_sleep': self.avg_sleep,
            'avg_mood': self.avg_mood,
            'total_calories': self.total_calories,
            'avg_daily_calories': self.avg_daily_calories,
            'created_at': self.created_at
        }

class WeeklyRecommendations:
    """Store recommendations given for a specific week"""

    def __init__(self, user_id: str, week_start: str, recommendations: Dict):
        self.user_id = user_id
        self.week_start = week_start
        self.recommendations = recommendations
        self.created_at = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # Track user response to recommendations
        self.user_feedback = {
            'followed_exercise': None,
            'followed_nutrition': None,
            'followed_sleep': None,
            'difficulty_level': None,  # 1-5 scale
            'effectiveness': None,     # 1-5 scale
            'notes': ""
        }

    def update_feedback(self, feedback: Dict):
        """Update user feedback on recommendations"""
        self.user_feedback.update(feedback)

    def to_dict(self):
        return {
            'user_id': self.user_id,
            'week_start': self.week_start,
            'recommendations': self.recommendations,
            'user_feedback': self.user_feedback,
            'created_at': self.created_at
        }

class HealthDataManager:
    """Manage all user health data and provide persistence"""

    def __init__(self, data_dir: str = "health_data"):
        self.data_dir = data_dir
        self._ensure_directories()

    def _ensure_directories(self):
        """Create necessary directories"""
        os.makedirs(f"{self.data_dir}/profiles", exist_ok=True)
        os.makedirs(f"{self.data_dir}/weekly_data", exist_ok=True)
        os.makedirs(f"{self.data_dir}/recommendations", exist_ok=True)

    def save_user_profile(self, profile: UserProfile):
        """Save user profile to file"""
        file_path = f"{self.data_dir}/profiles/{profile.user_id}.json"
        with open(file_path, 'w') as f:
            json.dump(profile.to_dict(), f, indent=2)

    def load_user_profile(self, user_id: str) -> Optional[UserProfile]:
        """Load user profile from file"""
        file_path = f"{self.data_dir}/profiles/{user_id}.json"
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                data = json.load(f)
                profile = UserProfile(user_id)
                profile.targets = data['targets']
                profile.preferences = data['preferences']
                profile.response_patterns = data['response_patterns']
                profile.created_date = data['created_date']
                return profile
        return None

    def save_weekly_data(self, weekly_data: WeeklyHealthData):
        """Save weekly health data"""
        user_dir = f"{self.data_dir}/weekly_data/{weekly_data.user_id}"
        os.makedirs(user_dir, exist_ok=True)

        file_path = f"{user_dir}/{weekly_data.week_start}.json"
        with open(file_path, 'w') as f:
            json.dump(weekly_data.to_dict(), f, indent=2)

    def load_user_weekly_data(self, user_id: str, num_weeks: int = 4) -> List[WeeklyHealthData]:
        """Load recent weekly data for a user"""
        user_dir = f"{self.data_dir}/weekly_data/{user_id}"
        if not os.path.exists(user_dir):
            return []

        # Get all weekly data files
        files = [f for f in os.listdir(user_dir) if f.endswith('.json')]
        files.sort(reverse=True)  # Most recent first

        weekly_data_list = []
        for file in files[:num_weeks]:
            file_path = f"{user_dir}/{file}"
            with open(file_path, 'r') as f:
                data = json.load(f)
                # Recreate WeeklyHealthData object
                weekly_data = WeeklyHealthData(
                    data['user_id'],
                    data['week_start'],
                    data # Pass the whole dictionary to __init__
                )
                weekly_data_list.append(weekly_data)

        return weekly_data_list

    def save_recommendations(self, recommendations: WeeklyRecommendations):
        """Save weekly recommendations"""
        user_dir = f"{self.data_dir}/recommendations/{recommendations.user_id}"
        os.makedirs(user_dir, exist_ok=True)

        file_path = f"{user_dir}/{recommendations.week_start}.json"
        with open(file_path, 'w') as f:
            json.dump(recommendations.to_dict(), f, indent=2)

    def load_user_recommendations(self, user_id: str, num_weeks: int = 4) -> List[WeeklyRecommendations]:
        """Load recent recommendations for a user"""
        user_dir = f"{self.data_dir}/recommendations/{user_id}"
        if not os.path.exists(user_dir):
            return []

        files = [f for f in os.listdir(user_dir) if f.endswith('.json')]
        files.sort(reverse=True)

        rec_list = []
        for file in files[:num_weeks]:
            file_path = f"{user_dir}/{file}"
            with open(file_path, 'r') as f:
                data = json.load(f)
                rec = WeeklyRecommendations(
                    data['user_id'],
                    data['week_start'],
                    data['recommendations']
                )
                rec.user_feedback = data.get('user_feedback', {}) # Handle missing feedback
                rec.created_at = data.get('created_at', datetime.now().strftime('%Y-%m-%d %H:%M:%S')) # Handle missing created_at
                rec_list.append(rec)

        return rec_list

    def get_user_progress_summary(self, user_id: str) -> Dict:
        """Get overall progress summary for a user"""
        weekly_data = self.load_user_weekly_data(user_id, num_weeks=8)

        if not weekly_data:
            return {"message": "No data available"}

        # Calculate trends
        weeks = len(weekly_data)

        # Steps trend
        steps_trend = [w.avg_daily_steps for w in reversed(weekly_data)]

        # Sleep trend
        sleep_trend = [w.avg_sleep for w in reversed(weekly_data)]

        # Zone minutes trend
        zone_trend = [w.zone_minutes for w in reversed(weekly_data)]

        # Mood trend
        mood_trend = [w.avg_mood for w in reversed(weekly_data)]

        return {
            'weeks_tracked': weeks,
            'steps_trend': steps_trend,
            'sleep_trend': sleep_trend,
            'zone_trend': zone_trend,
            'mood_trend': mood_trend,
            'latest_week': weekly_data[0].to_dict() if weekly_data else None
        }

# ========================================================================
# STEP 1 USAGE EXAMPLE
# ========================================================================

def example_usage():
    """Example of how to use the data management system"""

    # Initialize data manager
    data_manager = HealthDataManager()

    # Create or load user profile
    user_id = "user_123"
    profile = data_manager.load_user_profile(user_id)

    if not profile:
        # Create new user
        profile = UserProfile(user_id)
        profile.update_targets({
            'daily_steps': 12000,  # Custom target
            'weekly_zone_minutes': 180
        })
        profile.update_preferences({
            'preferred_exercises': ['running', 'yoga'],
            'food_restrictions': ['gluten-free']
        })
        data_manager.save_user_profile(profile)
        print(f"Created new user profile for {user_id}")
    else:
        print(f"Loaded existing profile for {user_id}")

    # Example weekly data (using food_entries list)
    week_data = {
        'total_steps': 68000,
        'zone_minutes': 140,
        'sleep_hours': [7.5, 6.8, 8.2, 7.0, 6.5, 8.5, 7.8],
        'exercise_sessions': [
            {'type': 'running', 'duration': 30, 'date': '2024-01-15'},
            {'type': 'yoga', 'duration': 45, 'date': '2024-01-17'}
        ],
        'mood_scores': [7, 6, 8, 7, 5, 8, 7],
        'food_entries': [ # Use a list of food entries
            {'category': 'fruits', 'quantity': 3, 'date': '2024-01-15', 'calories': 200},
            {'category': 'vegetables', 'quantity': 4, 'date': '2024-01-15', 'calories': 150},
            {'category': 'red_meat', 'quantity': 1, 'date': '2024-01-15', 'calories': 500},
            # ... add more food entries for the week
        ]
    }

    # Save weekly data
    weekly_health_data = WeeklyHealthData(user_id, "2024-01-15", week_data)
    data_manager.save_weekly_data(weekly_health_data)
    print("Saved weekly health data")

    # Example recommendations
    recommendations = {
        'exercise': "Increase zone minutes by adding 2x20min cardio sessions",
        'nutrition': "Add protein-rich snacks between meals",
        'sleep': "Maintain current sleep schedule, it's working well",
        'overall': "Focus on cardiovascular fitness this week"
    }

    weekly_recs = WeeklyRecommendations(user_id, "2024-01-15", recommendations)
    data_manager.save_recommendations(weekly_recs)
    print("Saved weekly recommendations")

    # Get progress summary
    progress = data_manager.get_user_progress_summary(user_id)
    print(f"Progress summary: {progress}")

    return data_manager

if __name__ == "__main__":
    # Test the system
    data_manager = example_usage()

#Analytics Engine

In [ ]:
# ========================================================================
# STEP 2: Analytics Engine (Colab Version)
# ========================================================================
import numpy as np
from typing import Dict, List, Tuple
from datetime import datetime
import statistics

class HealthAnalytics:
    """Core analytics engine for health data analysis"""

    def __init__(self, data_manager):
        self.data_manager = data_manager

    def analyze_weekly_performance(self, user_id: str, current_week_data) -> Dict:
        """Comprehensive analysis of current week vs targets and trends"""

        # Load user profile and historical data
        profile = self.data_manager.load_user_profile(user_id)
        historical_data = self.data_manager.load_user_weekly_data(user_id, num_weeks=4)

        if not profile:
            return {"error": "User profile not found"}

        # Current week performance vs targets
        performance = self._calculate_target_performance(current_week_data, profile.targets)

        # Trend analysis
        trends = self._analyze_trends(current_week_data, historical_data)

        # Progress analysis
        progress = self._analyze_progress(current_week_data, historical_data)

        # Priority areas (what needs most attention)
        priorities = self._identify_priority_areas(performance, trends)

        #Food-specific analysis
        food_analysis = self._analyze_food_patterns(current_week_data, profile.targets)

        return {
            'user_id': user_id,
            'week_start': current_week_data.week_start,
            'performance': performance,
            'trends': trends,
            'progress': progress,
            'priorities': priorities,
            'food_analysis': food_analysis,
            'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

    def _calculate_target_performance(self, current_data, targets: Dict) -> Dict:
        """Calculate performance against targets"""

        performance = {}

        # Steps performance
        steps_achievement = (current_data.avg_daily_steps / targets['daily_steps']) * 100
        performance['steps'] = {
            'current': current_data.avg_daily_steps,
            'target': targets['daily_steps'],
            'achievement_percent': min(steps_achievement, 150),  # Cap at 150%
            'status': self._get_performance_status(steps_achievement)
        }

        # Zone minutes performance
        zone_achievement = (current_data.zone_minutes / targets['weekly_zone_minutes']) * 100
        performance['zone_minutes'] = {
            'current': current_data.zone_minutes,
            'target': targets['weekly_zone_minutes'],
            'achievement_percent': min(zone_achievement, 150),
            'status': self._get_performance_status(zone_achievement)
        }

        # Sleep performance
        sleep_achievement = (current_data.avg_sleep / targets['daily_sleep_hours']) * 100
        performance['sleep'] = {
            'current': current_data.avg_sleep,
            'target': targets['daily_sleep_hours'],
            'achievement_percent': min(sleep_achievement, 120),  # Sleep shouldn't be too much over
            'status': self._get_performance_status(sleep_achievement)
        }

        # Exercise sessions
        exercise_achievement = (len(current_data.exercise_sessions) / targets['weekly_exercise_sessions']) * 100
        performance['exercise_sessions'] = {
            'current': len(current_data.exercise_sessions),
            'target': targets['weekly_exercise_sessions'],
            'achievement_percent': min(exercise_achievement, 150),
            'status': self._get_performance_status(exercise_achievement)
        }

        food_categories = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']

        # Food category performance
        food_categories = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']

        for category in food_categories:
            weekly_key = f'weekly_{category}'
            if weekly_key in targets and weekly_key in current_data.weekly_food_totals:
                current_intake = current_data.weekly_food_totals[weekly_key]
                target_intake = targets[weekly_key]

                if target_intake > 0:
                    achievement = (current_intake / target_intake) * 100
                    performance[category] = {
                        'current': current_intake,
                        'target': target_intake,
                        'achievement_percent': min(achievement, 150),
                        'status': self._get_food_performance_status(achievement, category)
                    }

        # Overall performance score (including food categories)
        all_scores = []

        # Core metrics (weighted more heavily)
        core_metrics = ['steps', 'zone_minutes', 'sleep', 'exercise_sessions']
        for metric in core_metrics:
            if metric in performance:
                all_scores.extend([performance[metric]['achievement_percent']] * 2)  # Double weight

        # Food metrics
        for category in food_categories:
            if category in performance:
                all_scores.append(performance[category]['achievement_percent'])

        performance['overall'] = {
            'score': np.mean(all_scores) if all_scores else 0,
            'status': self._get_performance_status(np.mean(all_scores) if all_scores else 0)
        }

        return performance

        # Calories (if target is set)
        if targets.get('daily_calories'):
            calorie_achievement = (current_data.avg_daily_calories / targets['daily_calories']) * 100
            performance['calories'] = {
                'current': current_data.avg_daily_calories,
                'target': targets['daily_calories'],
                'achievement_percent': calorie_achievement,
                'status': self._get_calorie_status(calorie_achievement)
            }

        # Overall performance score
        scores = [perf['achievement_percent'] for perf in performance.values()]
        performance['overall'] = {
            'score': np.mean(scores),
            'status': self._get_performance_status(np.mean(scores))
        }

        return performance

    def _analyze_food_patterns(self, current_data, targets: Dict) -> Dict:
        """Analyze food intake patterns and provide insights"""

        food_analysis = {
            'balanced_diet_score': 0,
            'deficient_categories': [],
            'excess_categories': [],
            'dietary_insights': []
        }

        food_categories = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']
        category_scores = []

        for category in food_categories:
            weekly_key = f'weekly_{category}'
            if weekly_key in targets and weekly_key in current_data.weekly_food_totals:
                current_intake = current_data.weekly_food_totals[weekly_key]
                target_intake = targets[weekly_key]

                if target_intake > 0:
                    achievement_ratio = current_intake / target_intake
                    category_scores.append(min(achievement_ratio, 1.5))  # Cap at 150%

                    # Identify deficiencies and excesses
                    if achievement_ratio < 0.7:  # Less than 70% of target
                        food_analysis['deficient_categories'].append({
                            'category': category,
                            'current': current_intake,
                            'target': target_intake,
                            'deficit': target_intake - current_intake
                        })
                    elif achievement_ratio > 1.3:  # More than 130% of target
                        food_analysis['excess_categories'].append({
                            'category': category,
                            'current': current_intake,
                            'target': target_intake,
                            'excess': current_intake - target_intake
                        })

        # Calculate balanced diet score
        if category_scores:
            food_analysis['balanced_diet_score'] = (np.mean(category_scores) * 100)

        # Generate dietary insights
        if food_analysis['deficient_categories']:
            top_deficiency = max(food_analysis['deficient_categories'], key=lambda x: x['deficit'])
            food_analysis['dietary_insights'].append(f"Increase {top_deficiency['category']} intake by {top_deficiency['deficit']} servings/week")

        if food_analysis['excess_categories']:
            top_excess = max(food_analysis['excess_categories'], key=lambda x: x['excess'])
            food_analysis['dietary_insights'].append(f"Consider reducing {top_excess['category']} intake")

        # Check for dietary balance
        fruit_veg_total = current_data.weekly_food_totals.get('weekly_fruits', 0) + current_data.weekly_food_totals.get('weekly_vegetables', 0)
        if fruit_veg_total < 35:  # Less than 5 servings per day
            food_analysis['dietary_insights'].append("Increase fruit and vegetable intake for better nutrition")

        protein_sources = current_data.weekly_food_totals.get('weekly_meat', 0) + current_data.weekly_food_totals.get('weekly_legumes', 0)
        if protein_sources < 10:
            food_analysis['dietary_insights'].append("Ensure adequate protein intake from various sources")

        return food_analysis

    def _get_food_performance_status(self, achievement_percent: float, category: str) -> str:
        """Get performance status for food categories"""

        # Different thresholds for different food types
        if category in ['fruits', 'vegetables', 'water_glasses']:
            # For healthy foods, more is generally better
            if achievement_percent >= 90:
                return "excellent"
            elif achievement_percent >= 70:
                return "good"
            elif achievement_percent >= 50:
                return "fair"
            else:
                return "needs_improvement"
        else:
            # For other foods, balance is key
            if 80 <= achievement_percent <= 120:
                return "excellent"
            elif 60 <= achievement_percent <= 140:
                return "good"
            elif 40 <= achievement_percent <= 160:
                return "fair"
            else:
                return "needs_adjustment"

    def _get_performance_status(self, achievement_percent: float) -> str:
        """Get performance status based on achievement percentage"""
        if achievement_percent >= 100:
            return "excellent"
        elif achievement_percent >= 85:
            return "good"
        elif achievement_percent >= 70:
            return "fair"
        else:
            return "needs_improvement"

    def _analyze_trends(self, current_data, historical_data: List) -> Dict:
        """Analyze trends over time"""

        if len(historical_data) < 2:
            return {"message": "Insufficient data for trend analysis"}

        # Include current week in analysis
        all_data = [current_data] + historical_data
        all_data.sort(key=lambda x: x.week_start)  # Sort chronologically

        trends = {}

        # Steps trend
        steps_data = [w.avg_daily_steps for w in all_data]
        trends['steps'] = self._calculate_trend(steps_data, "steps/day")

        # Zone minutes trend
        zone_data = [w.zone_minutes for w in all_data]
        trends['zone_minutes'] = self._calculate_trend(zone_data, "minutes/week")

        # Sleep trend
        sleep_data = [w.avg_sleep for w in all_data]
        trends['sleep'] = self._calculate_trend(sleep_data, "hours/night")

        # Mood trend
        mood_data = [w.avg_mood for w in all_data if w.avg_mood > 0]
        if mood_data:
            trends['mood'] = self._calculate_trend(mood_data, "mood score")

        # Food trends
        food_categories = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']
        for category in food_categories:
            weekly_key = f'weekly_{category}'
            category_data = []
            for w in all_data:
                if hasattr(w, 'weekly_food_totals') and weekly_key in w.weekly_food_totals:
                    category_data.append(w.weekly_food_totals[weekly_key])

            if len(category_data) >= 2:
                trends[category] = self._calculate_trend(category_data, f"servings/week")

        return trends

    def _calculate_trend(self, data: List[float], unit: str) -> Dict:
        """Calculate trend direction and magnitude"""
        if len(data) < 2:
            return {"direction": "insufficient_data"}

        # Calculate linear trend
        x = np.arange(len(data))
        slope, intercept = np.polyfit(x, data, 1)

        # Determine trend direction
        if slope > 0.1:
            direction = "improving"
        elif slope < -0.1:
            direction = "declining"
        else:
            direction = "stable"

        # Calculate percentage change from first to last
        if data[0] != 0:
            percent_change = ((data[-1] - data[0]) / data[0]) * 100
        else:
            percent_change = 0

        return {
            'direction': direction,
            'slope': slope,
            'percent_change': percent_change,
            'recent_value': data[-1],
            'unit': unit,
            'data_points': len(data)
        }

    def _analyze_progress(self, current_data, historical_data: List) -> Dict:
        """Analyze progress compared to previous weeks"""

        if not historical_data:
            return {"message": "No historical data for comparison"}

        # Compare with last week
        last_week = historical_data[0]  # Most recent week

        progress = {}

        # Steps progress
        steps_change = current_data.avg_daily_steps - last_week.avg_daily_steps
        progress['steps'] = {
            'change': steps_change,
            'percent_change': (steps_change / last_week.avg_daily_steps) * 100 if last_week.avg_daily_steps > 0 else 0,
            'direction': 'improved' if steps_change > 0 else 'declined' if steps_change < 0 else 'maintained'
        }

        # Food progress
        food_categories = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']
        for category in food_categories:
            weekly_key = f'weekly_{category}'
            if (hasattr(current_data, 'weekly_food_totals') and weekly_key in current_data.weekly_food_totals and
                hasattr(last_week, 'weekly_food_totals') and weekly_key in last_week.weekly_food_totals):

                current_intake = current_data.weekly_food_totals[weekly_key]
                last_intake = last_week.weekly_food_totals[weekly_key]
                change = current_intake - last_intake

                progress[category] = {
                    'change': change,
                    'percent_change': (change / last_intake) * 100 if last_intake > 0 else 0,
                    'direction': 'improved' if change > 0 else 'declined' if change < 0 else 'maintained'
                }

        return progress


        # Zone minutes progress
        zone_change = current_data.zone_minutes - last_week.zone_minutes
        progress['zone_minutes'] = {
            'change': zone_change,
            'percent_change': (zone_change / last_week.zone_minutes) * 100 if last_week.zone_minutes > 0 else 0,
            'direction': 'improved' if zone_change > 0 else 'declined' if zone_change < 0 else 'maintained'
        }

        # Sleep progress
        sleep_change = current_data.avg_sleep - last_week.avg_sleep
        progress['sleep'] = {
            'change': sleep_change,
            'percent_change': (sleep_change / last_week.avg_sleep) * 100 if last_week.avg_sleep > 0 else 0,
            'direction': 'improved' if sleep_change > 0 else 'declined' if sleep_change < 0 else 'maintained'
        }

        # Mood progress
        if current_data.avg_mood > 0 and last_week.avg_mood > 0:
            mood_change = current_data.avg_mood - last_week.avg_mood
            progress['mood'] = {
                'change': mood_change,
                'percent_change': (mood_change / last_week.avg_mood) * 100,
                'direction': 'improved' if mood_change > 0 else 'declined' if mood_change < 0 else 'maintained'
            }

        return progress

    def _identify_priority_areas(self, performance: Dict, trends: Dict) -> List[Dict]:
        """Identify which areas need most attention"""

        priorities = []

        # Check each area
        for area, perf in performance.items():
            if area == 'overall':
                continue

            priority_score = 0
            reasons = []

            # Low performance
            if perf['achievement_percent'] < 70:
                priority_score += 3
                reasons.append(f"Below target ({perf['achievement_percent']:.1f}%)")
            elif perf['achievement_percent'] < 85:
                priority_score += 2
                reasons.append(f"Slightly below target ({perf['achievement_percent']:.1f}%)")

            # Declining trend
            if area in trends and trends[area].get('direction') == 'declining':
                priority_score += 2
                reasons.append("Declining trend")

            # Add to priorities if score is significant
            if priority_score > 0:
                priorities.append({
                    'area': area,
                    'priority_score': priority_score,
                    'reasons': reasons,
                    'current_performance': perf['achievement_percent']
                })

        # Sort by priority score (highest first)
        priorities.sort(key=lambda x: x['priority_score'], reverse=True)

        return priorities


    def _get_performance_status(self, achievement_percent: float) -> str:
        """Get performance status based on achievement percentage"""
        if achievement_percent >= 100:
            return "excellent"
        elif achievement_percent >= 85:
            return "good"
        elif achievement_percent >= 70:
            return "fair"
        else:
            return "needs_improvement"

    def _get_calorie_status(self, achievement_percent: float) -> str:
        """Get calorie status (different logic as too many calories is bad)"""
        if 90 <= achievement_percent <= 110:
            return "excellent"
        elif 80 <= achievement_percent <= 120:
            return "good"
        elif 70 <= achievement_percent <= 130:
            return "fair"
        else:
            return "needs_adjustment"

    def generate_insights(self, analysis: Dict) -> List[str]:
        """Generate human-readable insights from analysis"""

        insights = []

        # Overall performance insight
        overall_score = analysis['performance']['overall']['score']
        if overall_score >= 90:
            insights.append("🎉 Excellent overall performance this week!")
        elif overall_score >= 75:
            insights.append("👍 Good overall performance with room for improvement")
        else:
            insights.append("⚠️ Several areas need attention this week")

        # Priority area insights
        priorities = analysis['priorities']
        if priorities:
            top_priority = priorities[0]
            insights.append(f"🎯 Top priority: {top_priority['area'].replace('_', ' ').title()}")

        # Trend insights - FIXED
        trends = analysis.get('trends', {})
        improving_areas = []
        declining_areas = []

        # Check if trends is a dictionary before iterating
        if isinstance(trends, dict):
            for area, trend in trends.items():
                # Check if trend is a dictionary (not a string)
                if isinstance(trend, dict):
                    if trend.get('direction') == 'improving':
                        improving_areas.append(area)
                    elif trend.get('direction') == 'declining':
                        declining_areas.append(area)
                elif isinstance(trend, str):
                    # Handle case where trend is just a string
                    if trend == 'improving':
                        improving_areas.append(area)
                    elif trend == 'declining':
                        declining_areas.append(area)

        if improving_areas:
            insights.append(f"📈 Improving: {', '.join(improving_areas)}")

        if declining_areas:
            insights.append(f"📉 Needs attention: {', '.join(declining_areas)}")

        # Progress insights - FIXED
        progress = analysis.get('progress', {})
        if isinstance(progress, dict):
            for area, prog in progress.items():
                if isinstance(prog, dict):
                    if prog.get('direction') == 'improved' and prog.get('percent_change', 0) > 10:
                        insights.append(f"🚀 Great improvement in {area}: +{prog['percent_change']:.1f}%")

        return insights

# ========================================================================
# CORRECTED USAGE EXAMPLE FOR COLAB
# ========================================================================

def example_analytics():
    """Example of using the analytics engine - CORRECTED FOR COLAB"""

    # Use the data manager and classes from the previous cell (Step 1)
    # No import needed since they're already defined in the notebook

    data_manager = HealthDataManager()
    analytics = HealthAnalytics(data_manager)

    # Example current week data
    current_week_data = WeeklyHealthData("user_123", "2024-01-22", {
        'total_steps': 72000,  # Improved from last week
        'zone_minutes': 160,   # Good improvement
        'sleep_hours': [7.8, 7.2, 8.1, 7.5, 7.0, 8.3, 7.9],
        'exercise_sessions': [
            {'type': 'running', 'duration': 35},
            {'type': 'yoga', 'duration': 45},
            {'type': 'cycling', 'duration': 40}
        ],
        'mood_scores': [8, 7, 8, 7, 6, 9, 8],
        'food_data': []  # Simplified for example
    })

    # Perform analysis
    analysis = analytics.analyze_weekly_performance("user_123", current_week_data)

    # Generate insights
    insights = analytics.generate_insights(analysis)

    print("=== WEEKLY HEALTH ANALYSIS ===")
    print(f"User: {analysis['user_id']}")
    print(f"Week: {analysis['week_start']}")
    print(f"Overall Score: {analysis['performance']['overall']['score']:.1f}/100")

    print("\n=== KEY INSIGHTS ===")
    for insight in insights:
        print(insight)

    print("\n=== PERFORMANCE BREAKDOWN ===")
    for area, perf in analysis['performance'].items():
        if area != 'overall':
            print(f"{area.replace('_', ' ').title()}: {perf['achievement_percent']:.1f}% ({perf['status']})")

    if analysis['priorities']:
        print("\n=== PRIORITY AREAS ===")
        for priority in analysis['priorities'][:3]:  # Top 3
            print(f"{priority['area'].replace('_', ' ').title()}: {', '.join(priority['reasons'])}")

    return analysis

# Test the analytics (optional - run this to test)
if __name__ == "__main__":
    analysis = example_analytics()

In [ ]:
# ========================================================================
# STEP 3: LLM Integration with Your Health Model
# ========================================================================
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch
from langchain.vectorstores import FAISS
# Updated import to fix deprecation warning
try:
    from langchain_huggingface import HuggingFaceEmbeddings
except ImportError:
    from langchain.embeddings import HuggingFaceEmbeddings
import re
from typing import Dict, List
import json

class HealthLLMProcessor:
    """Enhanced LLM processor for health analytics with memory and learning"""

    def __init__(self, base_model_id: str, adapter_path: str, vector_db_path: str, data_manager):
        self.data_manager = data_manager
        self._load_model(base_model_id, adapter_path)
        self._load_vector_db(vector_db_path)

    def _load_model(self, base_model_id: str, adapter_path: str):
        """Load your fine-tuned model"""
        print("Loading health LLM...")

        # Your exact configuration
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
        )

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # Load model with quantization
        self.model = AutoModelForCausalLM.from_pretrained(
            base_model_id,
            device_map="auto",
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            quantization_config=bnb_config,
            trust_remote_code=True
        )

        # Load the LoRA adapter
        self.model = PeftModel.from_pretrained(self.model, adapter_path)
        self.model.eval()
        print("✅ Model loaded successfully")

    def _load_vector_db(self, vector_db_path: str):
        """Load your vector database"""
        print("Loading vector database...")
        self.embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        self.vectorstore = FAISS.load_local(
            vector_db_path,
            embeddings=self.embedding_model,
            allow_dangerous_deserialization=True
        )
        print("✅ Vector database loaded successfully")

    def generate_weekly_recommendations(self, user_id: str, analysis_data: Dict) -> Dict:
        """Generate comprehensive weekly recommendations based on analysis"""

        # Get user context
        user_context = self._build_user_context(user_id)

        # Get relevant knowledge from vector DB
        knowledge_context = self._get_relevant_knowledge(analysis_data)

        # Build prompt with all context
        prompt = self._build_comprehensive_prompt(user_id, analysis_data, user_context, knowledge_context)

        # Generate recommendations
        raw_response = self._generate_response(prompt)

        # Parse and structure the response
        structured_recommendations = self._parse_recommendations(raw_response)

        # Learn from this interaction
        self._update_user_learning(user_id, analysis_data, structured_recommendations)

        return structured_recommendations

    def _build_user_context(self, user_id: str) -> str:
        """Build context about user's history and preferences"""

        # Get user profile
        profile = self.data_manager.load_user_profile(user_id)

        # Get recent recommendations and feedback
        recent_recs = self.data_manager.load_user_recommendations(user_id, num_weeks=2)

        # Get historical data for patterns
        historical_data = self.data_manager.load_user_weekly_data(user_id, num_weeks=4)

        context = f"USER PROFILE:\n"

        if profile:
            context += f"- Targets: {profile.targets}\n"
            context += f"- Preferences: {profile.preferences}\n"
            context += f"- Response Patterns: {profile.response_patterns}\n"

        if recent_recs:
            context += f"\nRECENT RECOMMENDATIONS:\n"
            for rec in recent_recs[:2]:  # Last 2 weeks
                context += f"Week {rec.week_start}:\n"
                for area, recommendation in rec.recommendations.items():
                    context += f"  {area}: {recommendation}\n"
                if rec.user_feedback.get('followed_exercise'):
                    context += f"  User followed exercise advice: {rec.user_feedback['followed_exercise']}\n"

        if historical_data:
            context += f"\nHISTORICAL PATTERNS:\n"
            # Calculate what user typically achieves
            avg_steps = sum([w.avg_daily_steps for w in historical_data]) / len(historical_data)
            avg_zone = sum([w.zone_minutes for w in historical_data]) / len(historical_data)
            context += f"- Typical daily steps: {avg_steps:.0f}\n"
            context += f"- Typical zone minutes: {avg_zone:.0f}\n"

        return context

    def _get_relevant_knowledge(self, analysis_data: Dict) -> str:
        """Get relevant knowledge from vector database based on priority areas"""

        priorities = analysis_data.get('priorities', [])

        if not priorities:
            # General health query
            search_query = "healthy lifestyle exercise nutrition sleep wellness"
        else:
            # Build query based on priority areas
            priority_areas = [p['area'] for p in priorities[:2]]  # Top 2 priorities
            search_query = " ".join(priority_areas) + " improvement recommendations health"

        # Search vector database
        relevant_docs = self.vectorstore.similarity_search(search_query, k=4)

        # Combine relevant knowledge
        knowledge = "\n---\n".join([doc.page_content for doc in relevant_docs])

        return knowledge[:2000]  # Limit context length

    def _build_comprehensive_prompt(self, user_id: str, analysis_data: Dict, user_context: str, knowledge_context: str) -> str:
        """Build comprehensive prompt with all context"""

        performance = analysis_data.get('performance', {})
        priorities = analysis_data.get('priorities', [])
        trends = analysis_data.get('trends', {})
        progress = analysis_data.get('progress', {})

        prompt = f"""<|system|>
You are an expert health and wellness AI assistant. You provide personalized recommendations based on comprehensive health data analysis.

IMPORTANT INSTRUCTIONS:
1. Provide recommendations in EXACTLY 4 sections: Food Recommendation, Physical Exercise, Sleep and Recovery, Overall Suggestion
2. Use bullet points with a), b), c), d) format
3. Base recommendations on the user's specific data, priorities, and historical patterns
4. Consider what has worked/not worked for this user before
5. Make recommendations progressive and achievable
6. End with "### End ###"

{user_context}

CURRENT WEEK ANALYSIS:
Overall Performance Score: {performance.get('overall', {}).get('score', 0):.1f}/100

PERFORMANCE BREAKDOWN:
- Steps: {performance.get('steps', {}).get('achievement_percent', 0):.1f}% of target
- Zone Minutes: {performance.get('zone_minutes', {}).get('achievement_percent', 0):.1f}% of target
- Sleep: {performance.get('sleep', {}).get('achievement_percent', 0):.1f}% of target
- Exercise Sessions: {performance.get('exercise_sessions', {}).get('achievement_percent', 0):.1f}% of target

PRIORITY AREAS NEEDING ATTENTION:
{self._format_priorities(priorities)}

TRENDS:
{self._format_trends(trends)}

PROGRESS FROM LAST WEEK:
{self._format_progress(progress)}

KNOWLEDGE BASE CONTEXT:
{knowledge_context}
<|/system|>

<|user|>
Based on my comprehensive health analysis above, provide personalized recommendations for the upcoming week. Focus on my priority areas while building on what has worked for me before.
<|/user|>

<|assistant|>
Response:
"""

        return prompt

    def _format_priorities(self, priorities: List[Dict]) -> str:
        """Format priority areas for prompt"""
        if not priorities:
            return "No major areas of concern - maintain current performance"

        formatted = ""
        for i, priority in enumerate(priorities[:3], 1):
            formatted += f"{i}. {priority['area'].replace('_', ' ').title()}: {', '.join(priority['reasons'])}\n"

        return formatted

    def _format_trends(self, trends: Dict) -> str:
        """Format trends for prompt"""
        if not trends:
            return "Insufficient data for trend analysis"

        formatted = ""
        for area, trend in trends.items():
            # Handle both string and dict trend values
            if isinstance(trend, dict):
                direction = trend.get('direction', 'stable')
                change = trend.get('percent_change', 0)
                formatted += f"- {area.replace('_', ' ').title()}: {direction}"
                if change != 0:
                    formatted += f" ({change:+.1f}%)"
            elif isinstance(trend, str):
                # Handle case where trend is just a string
                if trend in ['improving', 'declining', 'stable', 'insufficient_data']:
                    formatted += f"- {area.replace('_', ' ').title()}: {trend}"
                else:
                    formatted += f"- {area.replace('_', ' ').title()}: insufficient data"
            else:
                # Fallback for any other type
                formatted += f"- {area.replace('_', ' ').title()}: insufficient data"
            formatted += "\n"

        return formatted

    def _format_progress(self, progress: Dict) -> str:
        """Format progress for prompt"""
        if not progress:
            return "No previous week for comparison"

        formatted = ""
        for area, prog in progress.items():
            # Handle both string and dict progress values
            if isinstance(prog, dict):
                direction = prog.get('direction', 'maintained')
                change = prog.get('change', 0)
                formatted += f"- {area.replace('_', ' ').title()}: {direction}"
                if change != 0:
                    formatted += f" ({change:+.1f})"
            elif isinstance(prog, str):
                # Handle case where progress is just a string
                formatted += f"- {area.replace('_', ' ').title()}: {prog}"
            else:
                # Fallback
                formatted += f"- {area.replace('_', ' ').title()}: no data"
            formatted += "\n"

        return formatted

    def _generate_response(self, prompt: str) -> str:
        """Generate response using your model"""

        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(self.model.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=500,
                min_new_tokens=100,
                temperature=0.3,
                top_p=0.85,
                top_k=40,
                do_sample=True,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=[
                    self.tokenizer.encode("### End ###")[0] if "### End ###" in self.tokenizer.get_vocab() else self.tokenizer.eos_token_id,
                    self.tokenizer.eos_token_id
                ],
                early_stopping=True,
                no_repeat_ngram_size=3
            )

        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract only the assistant's response
        if "<|assistant|>" in response:
            generated_text = response.split("<|assistant|>")[-1].strip()
        else:
            generated_text = response[len(prompt):].strip()

        # Clean up the response
        generated_text = re.sub(r'<\|.*?\|>', '', generated_text)
        generated_text = generated_text.strip()

        return generated_text

    def _parse_recommendations(self, raw_response: str) -> Dict:
        """Parse the LLM response into structured recommendations"""

        sections = {
            'food_recommendation': '',
            'physical_exercise': '',
            'sleep_and_recovery': '',
            'overall_suggestion': '',
            'raw_response': raw_response
        }

        # Extract each section
        patterns = {
            'food_recommendation': r'1\.\s*Food Recommendation[:\s]*(.*?)(?=2\.\s*Physical Exercise|$)',
            'physical_exercise': r'2\.\s*Physical Exercise[:\s]*(.*?)(?=3\.\s*Sleep and Recovery|$)',
            'sleep_and_recovery': r'3\.\s*Sleep and Recovery[:\s]*(.*?)(?=4\.\s*Overall Suggestion|$)',
            'overall_suggestion': r'4\.\s*Overall Suggestion[:\s]*(.*?)(?=### End ###|$)'
        }

        for section, pattern in patterns.items():
            match = re.search(pattern, raw_response, re.DOTALL | re.IGNORECASE)
            if match:
                sections[section] = match.group(1).strip()

        return sections

    def _update_user_learning(self, user_id: str, analysis_data: Dict, recommendations: Dict):
        """Update user learning patterns based on new recommendations"""

        # This will help improve future recommendations
        profile = self.data_manager.load_user_profile(user_id)

        if profile:
            # Track recommendation types given
            priorities = analysis_data.get('priorities', [])
            if priorities:
                top_priority = priorities[0]['area']
                if top_priority not in profile.response_patterns.get('areas_worked_on', []):
                    if 'areas_worked_on' not in profile.response_patterns:
                        profile.response_patterns['areas_worked_on'] = []
                    profile.response_patterns['areas_worked_on'].append(top_priority)

            # Save updated profile
            self.data_manager.save_user_profile(profile)

    def update_recommendation_feedback(self, user_id: str, week_start: str, feedback: Dict):
        """Update feedback on recommendations to improve future suggestions"""

        # Load the recommendation
        recommendations = self.data_manager.load_user_recommendations(user_id, num_weeks=4)

        for rec in recommendations:
            if rec.week_start == week_start:
                rec.update_feedback(feedback)
                self.data_manager.save_recommendations(rec)

                # Update user profile learning patterns
                profile = self.data_manager.load_user_profile(user_id)
                if profile:
                    # Update response patterns based on feedback
                    if feedback.get('followed_exercise'):
                        profile.response_patterns['follows_exercise_recs'] = min(1.0,
                            profile.response_patterns.get('follows_exercise_recs', 0.5) + 0.1)

                    if feedback.get('followed_nutrition'):
                        profile.response_patterns['follows_nutrition_recs'] = min(1.0,
                            profile.response_patterns.get('follows_nutrition_recs', 0.5) + 0.1)

                    if feedback.get('followed_sleep'):
                        profile.response_patterns['follows_sleep_recs'] = min(1.0,
                            profile.response_patterns.get('follows_sleep_recs', 0.5) + 0.1)

                    self.data_manager.save_user_profile(profile)
                break

# ========================================================================
# STEP 3 USAGE EXAMPLE (COLAB FIXED)
# ========================================================================

def example_llm_processing():
    """Example of using the enhanced LLM processor - CORRECTED FOR COLAB"""

    # No imports needed - classes are already defined in previous cells

    # Initialize components
    data_manager = HealthDataManager()
    analytics = HealthAnalytics(data_manager)

    # Create user profile first
    user_id = "user_123"
    profile = UserProfile(user_id)
    profile.update_targets({
        'daily_steps': 10000,
        'weekly_zone_minutes': 150,
        'daily_sleep_hours': 8,
        'weekly_exercise_sessions': 3,
        'daily_calories': 2000
    })
    data_manager.save_user_profile(profile)
    print(f"✅ Created user profile for {user_id}")

    # Create some historical data for better analysis
    historical_week = WeeklyHealthData(user_id, "2024-01-22", {
        'total_steps': 62000,
        'zone_minutes': 100,
        'sleep_hours': [6.8, 6.0, 7.5, 6.5, 6.2, 8.0, 7.2],
        'exercise_sessions': [{'type': 'walking', 'duration': 30}],
        'mood_scores': [6, 5, 7, 6, 4, 7, 6],
        'food_data': []
    })
    data_manager.save_weekly_data(historical_week)
    print("✅ Created historical data")

    try:
        # Initialize LLM processor with your model paths
        llm_processor = HealthLLMProcessor(
            base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B",
            adapter_path="AnjaliNV/WellBeing_LLM",
            vector_db_path="/content/drive/MyDrive/rag_index",  # Your vector DB path
            data_manager=data_manager
        )
        print("✅ LLM processor initialized")

        # Current week data
        current_week_data = WeeklyHealthData(user_id, "2024-01-29", {
            'total_steps': 58000,  # Below target
            'zone_minutes': 90,    # Below target
            'sleep_hours': [6.5, 6.2, 7.8, 6.8, 6.0, 8.2, 7.5],
            'exercise_sessions': [
                {'type': 'yoga', 'duration': 45}
            ],
            'mood_scores': [6, 5, 7, 6, 5, 8, 7],
            'food_data': []
        })
        data_manager.save_weekly_data(current_week_data)
        print("✅ Created current week data")

        # Analyze the week
        analysis = analytics.analyze_weekly_performance(user_id, current_week_data)
        print("✅ Analysis completed")

        # Check for analysis errors
        if 'error' in analysis:
            print(f"❌ Analysis error: {analysis['error']}")
            return None

        # Generate recommendations
        print("🤖 Generating recommendations with LLM...")
        recommendations = llm_processor.generate_weekly_recommendations(user_id, analysis)
        print("✅ Recommendations generated")

        print("\n" + "="*60)
        print("=== COMPREHENSIVE WEEKLY RECOMMENDATIONS ===")
        print("="*60)
        print(f"👤 User: {user_id}")
        print(f"📅 Week: {current_week_data.week_start}")
        print(f"🎯 Overall Score: {analysis['performance']['overall']['score']:.1f}/100")

        print("\n📊 PRIORITY AREAS:")
        if analysis['priorities']:
            for priority in analysis['priorities'][:2]:
                print(f"• {priority['area'].replace('_', ' ').title()}: {', '.join(priority['reasons'])}")
        else:
            print("• No major priority areas - maintain current performance")

        print("\n🍎 FOOD RECOMMENDATIONS:")
        print(recommendations.get('food_recommendation', 'No specific food recommendations available'))

        print("\n💪 EXERCISE RECOMMENDATIONS:")
        print(recommendations.get('physical_exercise', 'No specific exercise recommendations available'))

        print("\n😴 SLEEP & RECOVERY:")
        print(recommendations.get('sleep_and_recovery', 'No specific sleep recommendations available'))

        print("\n🎯 OVERALL SUGGESTIONS:")
        print(recommendations.get('overall_suggestion', 'No specific overall suggestions available'))

        # Save the recommendations
        weekly_recs = WeeklyRecommendations(user_id, current_week_data.week_start, {
            'food': recommendations.get('food_recommendation', ''),
            'exercise': recommendations.get('physical_exercise', ''),
            'sleep': recommendations.get('sleep_and_recovery', ''),
            'overall': recommendations.get('overall_suggestion', '')
        })

        data_manager.save_recommendations(weekly_recs)
        print("\n✅ Recommendations saved for future learning")

        return recommendations

    except Exception as e:
        print(f"❌ Error during LLM processing: {str(e)}")
        import traceback
        traceback.print_exc()

        # Return a fallback recommendation structure
        return {
            'food_recommendation': 'Error generating recommendations - please try again',
            'physical_exercise': 'Error generating recommendations - please try again',
            'sleep_and_recovery': 'Error generating recommendations - please try again',
            'overall_suggestion': 'Error generating recommendations - please try again',
            'error': str(e)
        }

# Test the LLM processing (optional - uncomment to test)
# if __name__ == "__main__":
#     recommendations = example_llm_processing()

In [ ]:
# ========================================================================
# SYSTEM THAT ACTUALLY USES YOUR LLM FOR RECOMMENDATIONS
# ========================================================================

import torch
import gc
from datetime import datetime

class WorkingLLMHealthSystem:
    """Health system that actually loads and uses your LLM"""

    def __init__(self, base_model_id: str, adapter_path: str, vector_db_path: str, data_dir: str = "health_data"):
        """Initialize system with working LLM"""

        print("🚀 Initializing Health System with LLM...")

        # Initialize basic components first
        self.data_manager = HealthDataManager(data_dir)
        self.analytics = HealthAnalytics(self.data_manager)
        self.UserProfile = UserProfile
        self.WeeklyHealthData = WeeklyHealthData
        self.WeeklyRecommendations = WeeklyRecommendations

        # Clear GPU memory first
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()
            print(f"🧹 GPU memory cleared. Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")

        # Load LLM components
        self.model = None
        self.tokenizer = None
        self.vectorstore = None

        self._load_llm_components(base_model_id, adapter_path, vector_db_path)

    def _load_llm_components(self, base_model_id: str, adapter_path: str, vector_db_path: str):
        """Load LLM components step by step"""

        try:
            # Step 1: Load vector database (lightweight)
            print("📚 Loading vector database...")
            try:
                from langchain_huggingface import HuggingFaceEmbeddings
            except ImportError:
                from langchain.embeddings import HuggingFaceEmbeddings

            from langchain.vectorstores import FAISS

            embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
            self.vectorstore = FAISS.load_local(
                vector_db_path,
                embeddings=embedding_model,
                allow_dangerous_deserialization=True
            )
            print("✅ Vector database loaded successfully")

        except Exception as e:
            print(f"⚠️ Vector database failed: {str(e)}")
            self.vectorstore = None

        try:
            # Step 2: Load tokenizer (lightweight)
            print("🔤 Loading tokenizer...")
            from transformers import AutoTokenizer

            self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer loaded successfully")

        except Exception as e:
            print(f"❌ Tokenizer loading failed: {str(e)}")
            return

        try:
            # Step 3: Load model (heavy - with optimizations)
            print("🤖 Loading your fine-tuned model...")
            from transformers import AutoModelForCausalLM, BitsAndBytesConfig
            from peft import PeftModel

            # Aggressive quantization config
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float16,
                llm_int8_enable_fp32_cpu_offload=True
            )

            # Try to load with CPU offloading
            try:
                print("  🔄 Attempting GPU + CPU hybrid loading...")
                self.model = AutoModelForCausalLM.from_pretrained(
                    base_model_id,
                    device_map="auto",
                    torch_dtype=torch.float16,
                    low_cpu_mem_usage=True,
                    quantization_config=bnb_config,
                    trust_remote_code=True,
                    max_memory={0: "6GB", "cpu": "8GB"}  # Limit GPU usage
                )

                # Load LoRA adapter
                self.model = PeftModel.from_pretrained(self.model, adapter_path)
                self.model.eval()

                print("✅ Model loaded successfully with GPU + CPU hybrid!")

            except Exception as e:
                print(f"  ⚠️ Hybrid loading failed: {str(e)}")
                print("  🔄 Trying CPU-only loading...")

                # Fallback to CPU-only
                torch.cuda.empty_cache()
                gc.collect()

                self.model = AutoModelForCausalLM.from_pretrained(
                    base_model_id,
                    device_map="cpu",
                    torch_dtype=torch.float16,
                    low_cpu_mem_usage=True,
                    trust_remote_code=True
                )

                self.model = PeftModel.from_pretrained(self.model, adapter_path)
                self.model.eval()

                print("✅ Model loaded successfully on CPU!")

        except Exception as e:
            print(f"❌ Model loading completely failed: {str(e)}")
            self.model = None

    def process_weekly_data(self, user_id: str, week_data: Dict, targets: Dict = None) -> Dict:
        """Process data and generate LLM recommendations"""

        try:
            print(f"📊 Processing data for user: {user_id}")

            # Basic data processing (same as before)
            profile = self.data_manager.load_user_profile(user_id)
            if not profile:
                profile = self.UserProfile(user_id)
                if targets:
                    profile.update_targets(targets)
                self.data_manager.save_user_profile(profile)

            # Create simplified weekly data for analytics
            week_start = week_data.get('week_start', datetime.now().strftime('%Y-%m-%d'))
            simplified_data = {
                'week_start': week_start,
                'total_steps': week_data.get('total_steps', 0),
                'zone_minutes': week_data.get('zone_minutes', 0),
                'sleep_hours': week_data.get('sleep_hours', []),
                'mood_scores': week_data.get('mood_scores', []),
                'exercise_sessions': week_data.get('exercise_sessions', []),
                'food_data': []
            }

            weekly_health_data = self.WeeklyHealthData(user_id, week_start, simplified_data)
            self.data_manager.save_weekly_data(weekly_health_data)

            # Perform analysis
            analysis = self.analytics.analyze_weekly_performance(user_id, weekly_health_data)
            print("✅ Analysis completed")

            # Generate recommendations using LLM
            if self.model and self.tokenizer:
                print("🤖 Generating LLM recommendations...")
                recommendations = self._generate_llm_recommendations(analysis, week_data, profile)
                print("✅ LLM recommendations generated!")
            else:
                print("⚠️ LLM not available, using smart fallback...")
                recommendations = self._generate_smart_fallback(analysis, week_data)

            # Save and return results
            weekly_recs = self.WeeklyRecommendations(user_id, week_start, {
                'food': recommendations.get('food_recommendation', ''),
                'exercise': recommendations.get('physical_exercise', ''),
                'sleep': recommendations.get('sleep_and_recovery', ''),
                'overall': recommendations.get('overall_suggestion', ''),
            })

            self.data_manager.save_recommendations(weekly_recs)

            return {
                'user_id': user_id,
                'week_start': week_start,
                'analysis': analysis,
                'recommendations': recommendations,
                'insights': self.analytics.generate_insights(analysis),
                'week_summary': self._create_week_summary(weekly_health_data, week_data),
                'llm_used': self.model is not None  # Tell user if LLM was used
            }

        except Exception as e:
            print(f"❌ Error in process_weekly_data: {str(e)}")
            import traceback
            traceback.print_exc()
            return self._create_error_response(user_id, week_data, str(e))

    def _generate_llm_recommendations(self, analysis: Dict, week_data: Dict, profile) -> Dict:
        """Generate recommendations using your actual LLM"""

        try:
            # Build context for LLM
            context = self._build_llm_context(analysis, week_data, profile)

            # Create prompt for your model
            prompt = f"""<|system|>
You are a expert health and wellness advisor. Provide personalized recommendations based on the user's health data analysis.

IMPORTANT FORMAT:
Respond with exactly 4 sections in this format:
1. Food Recommendation:
2. Physical Exercise:
3. Sleep and Recovery:
4. Overall Suggestion:

Use bullet points with a), b), c), d) format for each section.
End with "### End ###"
<|/system|>

<|user|>
{context}

Based on this analysis, provide comprehensive health recommendations for the upcoming week.
<|/user|>

<|assistant|>
1. Food Recommendation:
"""

            print(f"📝 Sending prompt to LLM (length: {len(prompt)} chars)")

            # Generate with your model
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1500)

            # Move to same device as model
            if next(self.model.parameters()).device != inputs['input_ids'].device:
                inputs = {k: v.to(next(self.model.parameters()).device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=400,
                    min_new_tokens=100,
                    temperature=0.7,
                    top_p=0.9,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )

            # Decode response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = response[len(prompt):].strip()

            print(f"✅ LLM generated {len(response)} characters")
            print(f"📄 Response preview: {response[:200]}...")

            # Parse the response
            parsed_recommendations = self._parse_llm_response(response)

            # Add metadata
            parsed_recommendations['llm_generated'] = True
            parsed_recommendations['model_used'] = 'AnjaliNV/WellBeing_LLM'

            return parsed_recommendations

        except Exception as e:
            print(f"❌ LLM generation failed: {str(e)}")
            import traceback
            traceback.print_exc()
            return self._generate_smart_fallback(analysis, week_data)

    def _build_llm_context(self, analysis: Dict, week_data: Dict, profile) -> str:
        """Build comprehensive context for LLM"""

        performance = analysis.get('performance', {})
        priorities = analysis.get('priorities', [])

        # Build context string
        context_parts = []

        # Overall performance
        overall_score = performance.get('overall', {}).get('score', 0)
        context_parts.append(f"Overall Health Score: {overall_score:.1f}/100")

        # Performance breakdown
        perf_summary = []
        for area, perf in performance.items():
            if area != 'overall':
                achievement = perf.get('achievement_percent', 0)
                current = perf.get('current', 0)
                target = perf.get('target', 0)
                perf_summary.append(f"{area}: {current}/{target} ({achievement:.1f}%)")

        context_parts.append("Performance: " + ", ".join(perf_summary))

        # Priority areas
        if priorities:
            priority_list = [p['area'] for p in priorities[:3]]
            context_parts.append(f"Priority areas: {', '.join(priority_list)}")

        # Food data analysis
        food_data = week_data.get('food_data', {})
        if food_data:
            food_summary = []
            targets = {
                'dairy': 14, 'legumes': 5, 'meat': 7, 'fruits': 21,
                'vegetables': 35, 'grains': 28, 'nuts_seeds': 7, 'water_glasses': 56
            }

            for category, target in targets.items():
                if category in food_data:
                    daily_values = food_data[category]
                    current = sum(daily_values) if isinstance(daily_values, list) else daily_values
                    percentage = (current / target) * 100 if target > 0 else 0
                    food_summary.append(f"{category}: {current}/{target} ({percentage:.0f}%)")

            context_parts.append("Food intake: " + ", ".join(food_summary))

        # User preferences
        if hasattr(profile, 'preferences') and profile.preferences:
            restrictions = profile.preferences.get('food_restrictions', [])
            allergies = profile.preferences.get('food_allergies', [])
            if restrictions:
                context_parts.append(f"Dietary restrictions: {', '.join(restrictions)}")
            if allergies:
                context_parts.append(f"Food allergies: {', '.join(allergies)}")

        # Add knowledge base context if available
        if self.vectorstore:
            try:
                # Search for relevant information
                query = f"health recommendations {' '.join(priority_list[:2]) if priorities else 'wellness'}"
                docs = self.vectorstore.similarity_search(query, k=2)
                if docs:
                    context_parts.append("Relevant guidance: " + docs[0].page_content[:300])
            except:
                pass  # If vector search fails, continue without it

        return ". ".join(context_parts)

    def _parse_llm_response(self, response: str) -> Dict:
        """Parse LLM response into structured recommendations"""

        import re

        recommendations = {
            'food_recommendation': '',
            'physical_exercise': '',
            'sleep_and_recovery': '',
            'overall_suggestion': ''
        }

        # Try to extract sections using regex
        patterns = {
            'food_recommendation': r'1\.\s*Food Recommendation[:\s]*(.*?)(?=2\.\s*Physical Exercise|$)',
            'physical_exercise': r'2\.\s*Physical Exercise[:\s]*(.*?)(?=3\.\s*Sleep and Recovery|$)',
            'sleep_and_recovery': r'3\.\s*Sleep and Recovery[:\s]*(.*?)(?=4\.\s*Overall Suggestion|$)',
            'overall_suggestion': r'4\.\s*Overall Suggestion[:\s]*(.*?)(?=### End ###|$)'
        }

        for section, pattern in patterns.items():
            match = re.search(pattern, response, re.DOTALL | re.IGNORECASE)
            if match:
                content = match.group(1).strip()
                # Clean up content
                content = re.sub(r'\n+', '\n', content)
                content = content.strip()
                recommendations[section] = content

        # If parsing failed, try simpler approach
        if not any(recommendations.values()):
            # Split by numbers
            parts = re.split(r'\d+\.\s*(?:Food|Physical|Sleep|Overall)', response)
            if len(parts) >= 4:
                recommendations['food_recommendation'] = parts[1].strip() if len(parts) > 1 else ''
                recommendations['physical_exercise'] = parts[2].strip() if len(parts) > 2 else ''
                recommendations['sleep_and_recovery'] = parts[3].strip() if len(parts) > 3 else ''
                recommendations['overall_suggestion'] = parts[4].strip() if len(parts) > 4 else ''

        # Fallback: use entire response if still empty
        if not any(recommendations.values()):
            recommendations['overall_suggestion'] = response[:500] + "..." if len(response) > 500 else response

        return recommendations

    def _generate_smart_fallback(self, analysis: Dict, week_data: Dict) -> Dict:
        """Smart fallback recommendations when LLM fails"""

        return {
            'food_recommendation': 'LLM unavailable - Focus on balanced nutrition with variety from all food groups.',
            'physical_exercise': 'LLM unavailable - Continue regular exercise and aim for 150 zone minutes per week.',
            'sleep_and_recovery': 'LLM unavailable - Maintain 7-9 hours of quality sleep nightly.',
            'overall_suggestion': 'LLM unavailable - Keep up your healthy habits and focus on consistency.',
            'llm_generated': False
        }

    def _create_week_summary(self, weekly_data, week_data: Dict) -> str:
        """Create week summary"""
        summary = f"Week of {weekly_data.week_start}:\n"
        summary += f"• Steps: {weekly_data.total_steps:,} ({weekly_data.avg_daily_steps:.0f}/day)\n"
        summary += f"• Zone Minutes: {weekly_data.zone_minutes}\n"
        summary += f"• Average Sleep: {weekly_data.avg_sleep:.1f} hours\n"
        summary += f"• Exercise Sessions: {len(weekly_data.exercise_sessions)}\n"
        summary += f"• Average Mood: {weekly_data.avg_mood:.1f}/10\n"

        # Add LLM status
        if self.model:
            summary += "• LLM Status: ✅ Active (using your fine-tuned model)\n"
        else:
            summary += "• LLM Status: ❌ Unavailable (using rule-based recommendations)\n"

        return summary

    def _create_error_response(self, user_id: str, week_data: Dict, error: str) -> Dict:
        """Create error response"""
        return {
            'user_id': user_id,
            'week_start': week_data.get('week_start', 'unknown'),
            'analysis': {'performance': {'overall': {'score': 0}}, 'priorities': []},
            'recommendations': {
                'food_recommendation': f'Error: {error}',
                'physical_exercise': 'System error occurred.',
                'sleep_and_recovery': 'Please try again.',
                'overall_suggestion': 'Contact support if issue persists.'
            },
            'insights': ['System error during analysis'],
            'week_summary': f'Error: {error}',
            'llm_used': False
        }

    def provide_feedback(self, user_id: str, week_start: str, feedback: Dict):
        """Provide feedback"""
        print(f"✅ Feedback received for {user_id}")

    def get_user_progress(self, user_id: str, weeks: int = 8) -> Dict:
        """Get user progress"""
        try:
            progress_summary = self.data_manager.get_user_progress_summary(user_id)
            recent_recommendations = self.data_manager.load_user_recommendations(user_id, weeks)

            return {
                'progress_summary': progress_summary,
                'recent_recommendations': [rec.to_dict() for rec in recent_recommendations],
                'trend_analysis': {'steps_trend': 'stable', 'zone_trend': 'stable', 'sleep_trend': 'stable'}
            }
        except Exception as e:
            return {
                'progress_summary': {'message': f'Error: {str(e)}'},
                'recent_recommendations': [],
                'trend_analysis': {}
            }

# ========================================================================
# CREATE SYSTEM THAT ACTUALLY USES YOUR LLM
# ========================================================================

def create_llm_health_app():
    """Create health app that actually uses your LLM"""

    print("🚀 Creating Health App with LLM Integration...")

    # Create system with LLM
    health_system = WorkingLLMHealthSystem(
        base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B",
        adapter_path="AnjaliNV/WellBeing_LLM",
        vector_db_path="/content/drive/MyDrive/rag_index",
        data_dir="/content/health_data"
    )

    # Create interface
    interface = FixedHealthAnalyticsInterface(health_system)
    demo = interface.create_simple_working_interface()

    print("✅ LLM Health App created!")

    # Test LLM
    if health_system.model:
        print("🎉 Your LLM is loaded and ready!")
        print("✅ Recommendations will be generated by your fine-tuned model")
    else:
        print("⚠️ LLM failed to load - will use smart fallbacks")
        print("🔧 Check GPU memory or try restarting runtime")

    return demo, health_system

def test_llm_system():
    """Test the LLM system"""

    print("🧪 Testing LLM System...")

    try:
        demo, health_system = create_llm_health_app()

        # Test with data
        test_data = {
            'week_start': '2024-01-15',
            'total_steps': 55000,  # Below target
            'zone_minutes': 90,    # Below target
            'sleep_hours': [6.5, 6.0, 7.5, 6.8, 6.2, 8.0, 7.0],
            'mood_scores': [6, 5, 7, 6, 5, 7, 6],
            'exercise_sessions': [{'type': 'test', 'duration': 30}],
            'food_data': {
                'dairy': [1, 2, 1, 2, 1, 2, 1],      # 10/14 - low
                'fruits': [2, 2, 3, 2, 2, 3, 2],     # 16/21 - low
                'vegetables': [3, 4, 3, 4, 3, 4, 3], # 24/35 - low
                'water_glasses': [6, 7, 6, 7, 6, 7, 6] # 45/56 - low
            }
        }

        result = health_system.process_weekly_data("llm_test_user", test_data, {
            'daily_steps': 10000, 'weekly_zone_minutes': 150,
            'daily_sleep_hours': 8, 'weekly_exercise_sessions': 3
        })

        print(f"✅ Test completed!")
        print(f"🎯 Health Score: {result['analysis']['performance']['overall']['score']:.1f}/100")
        print(f"🤖 LLM Used: {'✅ Yes' if result.get('llm_used') else '❌ No'}")

        if result.get('llm_used'):
            print("🎉 SUCCESS: Your LLM generated the recommendations!")
        else:
            print("⚠️ LLM not used - check model loading")

        return demo, health_system

    except Exception as e:
        print(f"❌ Test failed: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

print("""
🤖 LLM-POWERED HEALTH SYSTEM READY!

This version actually loads and uses your fine-tuned model for recommendations.

To use:
>>> demo, health_system = create_llm_health_app()
>>> demo.launch(share=True)

To test LLM first:
>>> demo, health_system = test_llm_system()
>>> demo.launch(share=True)

Features:
✅ Loads your actual fine-tuned model (AnjaliNV/WellBeing_LLM)
✅ Uses advanced memory optimization for GPU constraints
✅ Generates recommendations with your LLM
✅ Falls back gracefully if LLM fails
✅ Shows LLM status in results
✅ Handles food categories in LLM context
✅ Proper prompt formatting for your model

The system will tell you if LLM is working:
- "LLM Status: ✅ Active" = Your model is generating recommendations
- "LLM Status: ❌ Unavailable" = Using fallback recommendations

Run the test to see if your LLM loads properly! 🚀
""")

# Uncomment to test:
# demo, health_system = test_llm_system()
# demo.launch(share=True)

In [ ]:
# Create and launch immediately
demo, health_system = create_llm_health_app()
demo.launch(share=True)